In [ ]:
!pip install nemo-toolkit[asr]

In [4]:
# ------------------------------
# 1. Funções auxiliares (ASR)
# ------------------------------
import librosa
import numpy as np
import time
import nemo.collections.asr as nemo_asr
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

def carregar_audio(caminho, sr=16000):
    """Carrega arquivo de áudio e converte para 16 kHz mono, retornando duração."""
    audio, _ = librosa.load(caminho, sr=sr, mono=True)
    return np.array(audio, dtype=np.float32), sr, len(audio) / sr  # duração em segundos

# ------------------------------
# Função para modelos NeMo
# ------------------------------
def avaliar_modelo_nemo(modelo_id, caminho_audio):
    """
    Executa inferência com modelos NeMo, retornando:
    - RTF
    - latência total (segundos)
    - transcrição
    """
    print(f"🔽 Carregando modelo NeMo {modelo_id} ...")
    asr_model = nemo_asr.models.ASRModel.from_pretrained(modelo_id)

    # Carregar áudio e obter duração
    audio, sr, duracao = carregar_audio(caminho_audio, sr=16000)

    inicio = time.time()
    transcricao_obj = asr_model.transcribe([caminho_audio])[0]
    fim = time.time()

    tempo_processamento = fim - inicio  # latência total em segundos
    rtf = tempo_processamento / duracao  # cálculo do RTF

    transcricao_texto = getattr(transcricao_obj, "text", str(transcricao_obj))
    return rtf, tempo_processamento, transcricao_texto

# ------------------------------
# Função para modelos Hugging Face
# ------------------------------
def avaliar_modelo_hf(modelo_id, caminho_audio):
    """
    Executa inferência com modelos Hugging Face, retornando:
    - RTF
    - latência total (segundos)
    - transcrição
    """
    print(f"🔽 Carregando modelo HF {modelo_id} ...")
    processor = Wav2Vec2Processor.from_pretrained(modelo_id)
    model = Wav2Vec2ForCTC.from_pretrained(modelo_id)
    model.eval()

    audio, sr, duracao = carregar_audio(caminho_audio, sr=16000)
    input_values = processor(audio, sampling_rate=sr, return_tensors="pt").input_values

    inicio = time.time()
    with torch.no_grad():
        logits = model(input_values).logits
    fim = time.time()

    tempo_processamento = fim - inicio
    rtf = tempo_processamento / duracao

    pred_ids = torch.argmax(logits, dim=-1)
    transcricao_texto = processor.batch_decode(pred_ids)[0]

    return rtf, tempo_processamento, transcricao_texto

# ------------------------------
# Função para calcular métricas
# ------------------------------
def calcular_metricas(modelo, rtf, latencia_total, saida, referencia=None, rank=None):
    """
    Retorna dicionário com:
    - Rank
    - Modelo
    - RTF
    - Latência total (s e ms)
    - Transcrição
    - WER aproximado
    """
    if not isinstance(saida, str) and hasattr(saida, 'text'):
        saida = saida.text
    elif not isinstance(saida, str):
        saida = str(saida)

    resultado = {
        "Rank": rank if rank is not None else "-",
        "Modelo": modelo,
        "RTF": round(rtf, 3),
        "Latência (s)": round(latencia_total, 3),
        "Latência (ms)": round(latencia_total * 1000, 2),
        "Transcrição": saida,
        "WER aproximado": None
    }

    if referencia:
        ref_tokens = referencia.lower().split()
        out_tokens = saida.lower().split()
        intersecao = len(set(ref_tokens) & set(out_tokens))
        wer_aprox = 1 - (intersecao / len(ref_tokens)) if ref_tokens else 1.0
        resultado["WER aproximado"] = round(wer_aprox, 3)

    return resultado


In [ ]:
# ------------------------------
# Avaliação de modelos ASR específicos (PT-BR) - com RTF + LATÊNCIA
# ------------------------------

import os
import time
import pandas as pd
import librosa
import nemo.collections.asr as nemo_asr
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import numpy as np

# ------------------------------
# Funções auxiliares
# ------------------------------

def carregar_audio(caminho, sr=16000):
    """Carrega arquivo de áudio, converte para 16 kHz mono e retorna duração em segundos."""
    audio, _ = librosa.load(caminho, sr=sr, mono=True)
    audio = np.array(audio, dtype=np.float32)
    duracao = len(audio) / sr if sr > 0 else 0.0
    return audio, sr, duracao

def avaliar_modelo_nemo(modelo_id, caminho_audio):
    """Executa inferência com modelos NeMo e retorna RTF, latência total e transcrição."""
    print(f"\n🔹 Avaliando modelo NeMo: {modelo_id}")
    asr_model = nemo_asr.models.ASRModel.from_pretrained(modelo_id)

    _, _, duracao = carregar_audio(caminho_audio, sr=16000)

    inicio = time.time()
    transcricao_obj = asr_model.transcribe([caminho_audio])[0]
    fim = time.time()

    tempo_processamento = fim - inicio
    rtf = tempo_processamento / duracao if duracao > 0 else float("inf")

    transcricao_texto = getattr(transcricao_obj, "text", str(transcricao_obj))
    return rtf, tempo_processamento, transcricao_texto

def avaliar_modelo_hf(modelo_id, caminho_audio):
    """Executa inferência com modelos Hugging Face e retorna RTF, latência total e transcrição."""
    print(f"\n🔹 Avaliando modelo HF: {modelo_id}")
    processor = Wav2Vec2Processor.from_pretrained(modelo_id)
    model = Wav2Vec2ForCTC.from_pretrained(modelo_id)
    model.eval()

    audio, sr, duracao = carregar_audio(caminho_audio)
    if sr != 16000:
        raise ValueError("O modelo HF requer áudio em 16 kHz.")

    inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)

    inicio = time.time()
    with torch.no_grad():
        logits = model(inputs.input_values).logits
    fim = time.time()

    tempo_processamento = fim - inicio
    rtf = tempo_processamento / duracao if duracao > 0 else float("inf")

    predicted_ids = torch.argmax(logits, dim=-1)
    transcricao_texto = processor.batch_decode(predicted_ids)[0]

    return rtf, tempo_processamento, transcricao_texto

def calcular_metricas(modelo, rtf, latencia_total, saida, referencia=None):
    """Retorna métricas básicas para comparação de transcrição (inclui RTF + latência total)."""
    resultado = {
        "Modelo": modelo,
        "RTF": round(rtf, 3),
        "Latência (s)": round(latencia_total, 3),
        "Latência (ms)": round(latencia_total * 1000, 2),
        "Transcrição": saida
    }

    if referencia:
        if not isinstance(saida, str):
            saida = str(saida)

        ref_tokens = referencia.lower().split()
        out_tokens = saida.lower().split()

        intersecao = len(set(ref_tokens) & set(out_tokens))
        wer_aprox = 1 - (intersecao / len(ref_tokens)) if ref_tokens else 1.0

        resultado["WER aproximado"] = round(wer_aprox, 3)

    return resultado

def limpar_transcricao(x):
    """Extrai texto se for objeto Hypothesis do NeMo."""
    try:
        if isinstance(x, (list, tuple)) and len(x) > 0:
            return getattr(x[0], "text", str(x[0]))
        return getattr(x, "text", str(x))
    except Exception:
        return str(x) if x is not None else ""

# ------------------------------
# Configurações
# ------------------------------

audio_teste = "/content/audio3.wav"
referencia_texto = "um dois três testando isto é um exemplo para a transcrição de áudio para texto"

if not os.path.isfile(audio_teste):
    raise FileNotFoundError(f"Arquivo de áudio não encontrado: {audio_teste}")

# ------------------------------
# Lista de modelos para testar
# ------------------------------
modelos = [
     {"nome": "FastConformer-Hybrid", "id": "nvidia/stt_pt_fastconformer_hybrid_large_pc", "tipo": "nemo"},
     {"nome": "Citrinet-PT-Gamma-0.25", "id": "neongeckocom/stt_pt_citrinet_512_gamma_0_25", "tipo": "nemo"},
     {"nome": "QuartzNet-PT-Ottema", "id": "ottema/stt_pt_quartznet15x5_ctc_small", "tipo": "nemo"},
     {"nome": "QuartzNet-PT", "id": "dominguesm/stt_pt_quartznet15x5_ctc_small", "tipo": "nemo"},
     {"nome": "Parakeet-TDT-0.6b-v3", "id": "nvidia/parakeet-tdt-0.6b-v3", "tipo": "nemo"},
     {"nome": "wav2vec2-PT-BR-Light", "id": "danielpedrozo/wav2vec2-portuguese-wpp-checkpoint-480", "tipo": "hf"},
]

# ------------------------------
# Execução e coleta de resultados
# ------------------------------
resultados = []

for modelo in modelos:
    try:
        if modelo["tipo"] == "nemo":
            rtf, latencia_total, transcricao = avaliar_modelo_nemo(modelo["id"], audio_teste)
        elif modelo["tipo"] == "hf":
            rtf, latencia_total, transcricao = avaliar_modelo_hf(modelo["id"], audio_teste)

        metricas = calcular_metricas(modelo["nome"], rtf, latencia_total, transcricao, referencia_texto)
        resultados.append(metricas)

    except Exception as e:
        print(f"❌ Erro ao avaliar modelo {modelo['nome']}: {e}")

# ------------------------------
# Criação do DataFrame
# ------------------------------
df_resultados = pd.DataFrame(resultados)

if "Transcrição" in df_resultados.columns:
    df_resultados["Transcrição"] = df_resultados["Transcrição"].apply(limpar_transcricao)

# -------- ORDENAÇÃO --------
colunas_ordem = [c for c in ["WER aproximado", "RTF"] if c in df_resultados.columns]

if colunas_ordem:
    df_resultados = df_resultados.sort_values(
        by=colunas_ordem,
        ascending=[True] * len(colunas_ordem),
        ignore_index=True
    )

# Adicionar coluna de ranking
df_resultados.insert(0, "Rank", range(1, len(df_resultados) + 1))

# -------- VISUALIZAÇÃO --------
pd.set_option("display.max_colwidth", None)

print("\n🔹 Resultados ordenados (melhor WER e menor RTF):")
print(df_resultados[["Rank", "Modelo", "WER aproximado", "RTF", "Latência (ms)", "Transcrição"]])

# -------- EXPORTAÇÃO --------
df_resultados.to_csv("resultados_asr.csv", index=False, encoding="utf-8")

with open("resultados_asr.txt", "w", encoding="utf-8") as f:
    for _, row in df_resultados.iterrows():
        f.write(f"Modelo: {row['Modelo']}\n")
        f.write(f"WER aproximado: {row.get('WER aproximado', 'N/A')}\n")
        f.write(f"RTF: {row['RTF']}\n")
        f.write(f"Latência (ms): {row['Latência (ms)']}\n")
        f.write(f"Transcrição: {row['Transcrição']}\n")
        f.write("-" * 60 + "\n")

print("\n✅ Resultados salvos em resultados_asr.csv e resultados_asr.txt")
